In [1]:
import time
import codecs
import ephem
from ephem import degree
import datetime
from datetime import timedelta
import urllib.request
import csv

In [5]:
# Load the Space Catalogue as a dictionary
satcat = "https://celestrak.com/pub/satcat.txt"
sat_catalogue = urllib.request.urlopen(satcat).read().decode('utf-8').strip().split("\n")

sat_dict = {}
for x in sat_catalogue:
    try:
        sat_dict[x.split()[1]] = x
    except IndexError:
        continue

In [6]:
import json
import csv
import datetime
from datetime import timedelta
import ephem

# Open the local TLE data file
local_tle_file = "update_tle.tsv"
output_file = "output/dataset.json"

# Read the local TLE data file
with open(local_tle_file, "r", encoding="utf-8") as file:
    reader = csv.reader(file, delimiter='\t')
    sheetValues = list(reader)

data_list = []

for i in range(2, len(sheetValues)):
    row = sheetValues[i]
    tle_line0 = row[0]
    tle_line1 = row[1]
    norad_id = row[2].replace('n_', '')
    possible_satellite_reentry_date = sat_dict[norad_id][75:85] + " " + "00:00:00"
    
    try:
        # Read TLE
        name = "sat_data"
        tle_rec = ephem.readtle(name, tle_line0, tle_line1)
        tle_rec.compute(possible_satellite_reentry_date)
        
        # Calculate the orbit until it reaches the threshold to reenter in the atmosphere - 122 km
        reentry_threshold = 122000  # official entry interface by ESA and NASA
        reentry_threshold = 50000   # limit of space orbit knudsen number
    
        if int(tle_rec.elevation) >= reentry_threshold:
            d = datetime.datetime.strptime(possible_satellite_reentry_date, '%Y-%m-%d %H:%M:%S')

            # Check whether the debris reenters after max two days 
            for i in range(0, 240):
                parseHour = d + timedelta(hours=i)
                tle_rec.compute(parseHour)

                if int(tle_rec.elevation) <= reentry_threshold:  # threshold of reentry   
                    data_list.append({
                        "tle_0": tle_line0,
                        "tle_1": tle_line1,
                        "lat": tle_rec.sublat / ephem.degree,
                        "lon": tle_rec.sublong / ephem.degree,
                        "norad_cat_num": norad_id,
                        "satellite_name": sat_dict[norad_id][23:47].strip(),
                        "ownership": sat_dict[norad_id][49:54].strip(),
                        "launch_date": sat_dict[norad_id][56:66].strip(),
                        "satellite_decay": sat_dict[norad_id][75:85].strip(),
                        "satellite_decay_hour": str(parseHour),
                        "rcs": sat_dict[norad_id][119:127].strip()
                    })
                    
                    next2Minutes = parseHour + timedelta(minutes=2)
                    tle_rec.compute(next2Minutes)
                    data_list.append({
                        "tle_0": tle_line0,
                        "tle_1": tle_line1,
                        "lat": tle_rec.sublat / ephem.degree,
                        "lon": tle_rec.sublong / ephem.degree,
                        "norad_cat_num": norad_id,
                        "satellite_name": sat_dict[norad_id][23:47].strip(),
                        "ownership": sat_dict[norad_id][49:54].strip(),
                        "launch_date": sat_dict[norad_id][56:66].strip(),
                        "satellite_decay": sat_dict[norad_id][75:85].strip(),
                        "satellite_decay_hour": str(next2Minutes),
                        "rcs": sat_dict[norad_id][119:127].strip()
                    })
                    
                    next6Minutes = parseHour + timedelta(minutes=6)
                    tle_rec.compute(next6Minutes)
                    data_list.append({
                        "tle_0": tle_line0,
                        "tle_1": tle_line1,
                        "lat": tle_rec.sublat / ephem.degree,
                        "lon": tle_rec.sublong / ephem.degree,
                        "norad_cat_num": norad_id,
                        "satellite_name": sat_dict[norad_id][23:47].strip(),
                        "ownership": sat_dict[norad_id][49:54].strip(),
                        "launch_date": sat_dict[norad_id][56:66].strip(),
                        "satellite_decay": sat_dict[norad_id][75:85].strip(),
                        "satellite_decay_hour": str(next6Minutes),
                        "rcs": sat_dict[norad_id][119:127].strip()
                    })
                    break

        elif int(tle_rec.elevation) <= reentry_threshold:  # threshold of reentry
            next2Minutes = parseHour + timedelta(minutes=2)
            tle_rec.compute(next2Minutes)
            data_list.append({
                "tle_0": tle_line0,
                "tle_1": tle_line1,
                "lat": tle_rec.sublat / ephem.degree,
                "lon": tle_rec.sublong / ephem.degree,
                "norad_cat_num": norad_id,
                "satellite_name": sat_dict[norad_id][23:47].strip(),
                "ownership": sat_dict[norad_id][49:54].strip(),
                "launch_date": sat_dict[norad_id][56:66].strip(),
                "satellite_decay": sat_dict[norad_id][75:85].strip(),
                "satellite_decay_hour": str(next2Minutes),
                "rcs": sat_dict[norad_id][119:127].strip()
            })

            next6Minutes = parseHour + timedelta(minutes=6)
            tle_rec.compute(next6Minutes)
            data_list.append({
                "tle_0": tle_line0,
                "tle_1": tle_line1,
                "lat": tle_rec.sublat / ephem.degree,
                "lon": tle_rec.sublong / ephem.degree,
                "norad_cat_num": norad_id,
                "satellite_name": sat_dict[norad_id][23:47].strip(),
                "ownership": sat_dict[norad_id][49:54].strip(),
                "launch_date": sat_dict[norad_id][56:66].strip(),
                "satellite_decay": sat_dict[norad_id][75:85].strip(),
                "satellite_decay_hour": str(next6Minutes),
                "rcs": sat_dict[norad_id][119:127].strip()
            })
    
    except Exception as e:
        continue

# Write data to minified JSON file
with open(output_file, "w", encoding="utf-8") as json_file:
    json.dump(data_list, json_file, ensure_ascii=False, separators=(',', ':'))

print("\nFinished parsing!")


Finished parsing!
